In [1]:
# potrzebne bilioteki
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import plotly.express as px
import pandas as pd

In [2]:
# wczytanie danych
df=pd.read_excel(r'C:\Users\natal\OneDrive\Dokumenty\AnalizaDanych\DATA\1.1.Wypadki_ogolem_smiertleni_ranni_POW.xlsx')
df_Wypadki=df.groupby(["Jednostka", "Rok"], as_index = False).sum()
# map emotions to a color
c = dict(zip(df_Wypadki["Jednostka"].unique(), px.colors.qualitative.G10))

In [3]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

index_page = html.Div([
    dcc.Link('Analiza wypadków', href='/page-1', className='button',style = {'marginBottom': '8px'}),
    html.Br(),
    dcc.Link('Go to Page 2', href='/page-2', className='button'),
])

page_1_layout = html.Div([
        html.H1('Analiza wypadków samochodowych w Polsce w latach 2014-2020'),
        html.Div([
        dcc.Dropdown(id='dpdn2', value=df_Wypadki['Jednostka'].unique(), multi=True,
                     options=[{'label': x, 'value': x} for x in
                              df_Wypadki.Jednostka.unique()]),
        html.Div([
            dcc.Graph(id='my-graph', figure={}, clickData=None, hoverData=None,  
                      config={
                          'staticPlot': False,     
                          'scrollZoom': True,     
                          'doubleClick': 'reset',  
                          'showTips': False,       
                          'displayModeBar': True,  
                          'watermark': True,},
                      className='' #six columns
                      ),
            #dcc.Graph(id='pie-graph', figure={}, className='six columns'),
            #dcc.Graph(id='pie-graph2', figure={}, className='six columns'),
        ]),
        html.Div([dcc.Graph(id='pie-graph', figure={}, className='six columns'),#six columns
            dcc.Graph(id='pie-graph2', figure={}, className='six columns')]) #six columns
    ]), 
    html.Div(id='page-1-content'),
    html.Br(),
    dcc.Link('Go to Page 2', href='/page-2'),
    html.Br(),
    dcc.Link('Go back to home', href='/')
])

page_2_layout = html.Div([
    html.H1('Page 2'),
    html.Div(id='page-2-content'),
    html.Br(),
    dcc.Link('Analiza wypadków', href='/page-1'),
    html.Br(),
    dcc.Link('Go back to home', href='/')
])

# Update the index
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-1':
        return page_1_layout
    elif pathname == '/page-2':
        return page_2_layout
    else:
        return index_page
    # You could also return a 404 "URL not found" page here

In [4]:
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    Input(component_id='dpdn2', component_property='value')
)

def update_graph(Jednostka_chosen):
    dff = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
    fig = px.line(data_frame=dff, x='Rok', y='Wypadki ogółem', color='Jednostka',
                  custom_data=['Jednostka', 'Wypadki ogółem', 'Ofiary śmiertelne', 'Ranni'],
                 title=f'Wypadki ogółem: ', color_discrete_map=c)
    fig.update_traces(mode='lines+markers')
    return fig



In [5]:
@app.callback(
    Output(component_id='pie-graph', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, Jednostka_chosen):
    if hov_data is None:
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        dff2 = df_Wypadki[df_Wypadki.Rok == 2014]
        print(dff2)
        fig2 = px.pie(data_frame=dff2, values='Ofiary śmiertelne', names='Jednostka',
                      title='Ofiary śmiertelne w 2014',color="Jednostka", color_discrete_map=c)
        return fig2
    else:
        print(f'hover data: {hov_data}')
        #print(hov_data['points'][0]['customdata'][0])
        #print(f'click data: {clk_data}')
        #print(f'selected data: {slct_data}')
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        hov_year = hov_data['points'][0]['x']
        dff2 = dff2[dff2.Rok == hov_year]
        fig2 = px.pie(data_frame=dff2, values='Ofiary śmiertelne', names='Jednostka',
                      title=f'Ofiary śmiertelne w: {hov_year}',color='Jednostka', color_discrete_map=c)
        return fig2
    
@app.callback(
    Output(component_id='pie-graph2', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, Jednostka_chosen):
    if hov_data is None:
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        dff2 = df_Wypadki[df_Wypadki.Rok == 2014]
        print(dff2)
        fig2 = px.pie(data_frame=dff2, values='Ranni', names='Jednostka',
                      title='Ranni w 2014',color='Jednostka', color_discrete_map=c)
        return fig2
    else:
        print(f'hover data: {hov_data}')
        #print(hov_data['points'][0]['customdata'][0])
        #print(f'click data: {clk_data}')
        #print(f'selected data: {slct_data}')
        dff2 = df_Wypadki[df_Wypadki.Jednostka.isin(Jednostka_chosen)]
        hov_year = hov_data['points'][0]['x']
        dff2 = dff2[dff2.Rok == hov_year]
        fig2 = px.pie(data_frame=dff2, values='Ranni', names='Jednostka',
                      title=f'Ranni w: {hov_year}',color='Jednostka', color_discrete_map=c)
        return fig2
        


In [ ]:

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Dec/2021 20:31:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 20:31:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 20:31:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 20:31:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 20:31:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 20:31:08] "POST /_dash-update-component HTTP/1.1" 200 -


debug 2 None
debug 1 None
               Jednostka   Rok  Wypadki ogółem  Ofiary śmiertelne  Ranni
0           DOLNOŚLĄSKIE  2014            2466                242   3154
7     KUJAWSKO-POMORSKIE  2014            1044                179   1112
14             LUBELSKIE  2014            1408                195   1667
21              LUBUSKIE  2014             717                 81    957
28           MAZOWIECKIE  2014            4385                518   5211
35           MAŁOPOLSKIE  2014            3936                234   4805
42              OPOLSKIE  2014             738                104    857
49          PODKARPACKIE  2014            1751                144   2184
56             PODLASKIE  2014             692                126    793
63             POMORSKIE  2014            2724                181   3476
70   WARMIŃSKO-MAZURSKIE  2014            1645                148   2052
77         WIELKOPOLSKIE  2014            2392                268   2808
84    ZACHODNIOPOMORSKIE 

127.0.0.1 - - [20/Dec/2021 20:31:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 20:31:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 20:31:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 20:31:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 20:31:15] "POST /_dash-update-component HTTP/1.1" 200 -


debug 2 {'points': [{'curveNumber': 1, 'pointNumber': 2, 'pointIndex': 2, 'x': 2016, 'y': 1018, 'customdata': ['KUJAWSKO-POMORSKIE', 1018, 178, 1101]}]}
hover data: {'points': [{'curveNumber': 1, 'pointNumber': 2, 'pointIndex': 2, 'x': 2016, 'y': 1018, 'customdata': ['KUJAWSKO-POMORSKIE', 1018, 178, 1101]}]}
debug 1 {'points': [{'curveNumber': 1, 'pointNumber': 2, 'pointIndex': 2, 'x': 2016, 'y': 1018, 'customdata': ['KUJAWSKO-POMORSKIE', 1018, 178, 1101]}]}
hover data: {'points': [{'curveNumber': 1, 'pointNumber': 2, 'pointIndex': 2, 'x': 2016, 'y': 1018, 'customdata': ['KUJAWSKO-POMORSKIE', 1018, 178, 1101]}]}


...

debug 2 {'points': [{'curveNumber': 3, 'pointNumber': 3, 'pointIndex': 3, 'x': 2017, 'y': 682, 'customdata': ['LUBUSKIE', 682, 84, 855]}]}
hover data: {'points': [{'curveNumber': 3, 'pointNumber': 3, 'pointIndex': 3, 'x': 2017, 'y': 682, 'customdata': ['LUBUSKIE', 682, 84, 855]}]}
debug 1 {'points': [{'curveNumber': 3, 'pointNumber': 3, 'pointIndex': 3, 'x': 2017, 'y': 682, 'customdata': ['LUBUSKIE', 682, 84, 855]}]}
hover data: {'points': [{'curveNumber': 3, 'pointNumber': 3, 'pointIndex': 3, 'x': 2017, 'y': 682, 'customdata': ['LUBUSKIE', 682, 84, 855]}]}


127.0.0.1 - - [20/Dec/2021 20:31:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 20:31:17] "POST /_dash-update-component HTTP/1.1" 200 -


debug 2 {'points': [{'curveNumber': 3, 'pointNumber': 4, 'pointIndex': 4, 'x': 2018, 'y': 663, 'customdata': ['LUBUSKIE', 663, 79, 790]}]}
hover data: {'points': [{'curveNumber': 3, 'pointNumber': 4, 'pointIndex': 4, 'x': 2018, 'y': 663, 'customdata': ['LUBUSKIE', 663, 79, 790]}]}
debug 1 {'points': [{'curveNumber': 3, 'pointNumber': 4, 'pointIndex': 4, 'x': 2018, 'y': 663, 'customdata': ['LUBUSKIE', 663, 79, 790]}]}
hover data: {'points': [{'curveNumber': 3, 'pointNumber': 4, 'pointIndex': 4, 'x': 2018, 'y': 663, 'customdata': ['LUBUSKIE', 663, 79, 790]}]}


127.0.0.1 - - [20/Dec/2021 20:31:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Dec/2021 20:31:18] "POST /_dash-update-component HTTP/1.1" 200 -


debug 2 {'points': [{'curveNumber': 1, 'pointNumber': 3, 'pointIndex': 3, 'x': 2017, 'y': 949, 'customdata': ['KUJAWSKO-POMORSKIE', 949, 151, 1080]}]}
hover data: {'points': [{'curveNumber': 1, 'pointNumber': 3, 'pointIndex': 3, 'x': 2017, 'y': 949, 'customdata': ['KUJAWSKO-POMORSKIE', 949, 151, 1080]}]}
debug 1 {'points': [{'curveNumber': 1, 'pointNumber': 3, 'pointIndex': 3, 'x': 2017, 'y': 949, 'customdata': ['KUJAWSKO-POMORSKIE', 949, 151, 1080]}]}
hover data: {'points': [{'curveNumber': 1, 'pointNumber': 3, 'pointIndex': 3, 'x': 2017, 'y': 949, 'customdata': ['KUJAWSKO-POMORSKIE', 949, 151, 1080]}]}
